In [7]:
# import libaries for data collection + database storage
from pymongo import MongoClient
import requests
import time

# for initial analysis of data
import pandas as pd

The NFT collection that we are looking at for this project is LostPoets by pak. The collection has 27,557(27.5K) items in it (called 'asset's and scraped as such) and as of writing, has a volume trade of 26.6K. 

**1_sales_data_collection_OpenSea** & **1_assets_data_collection_OpenSea** Notebooks use the 2 API's from the OpenSea NFT marketplace (assets & events APIs) in order to gather data and store them into MongoDB.

**1_assets_data_collection_OpenSea** also uses another API to gather data on the whole of the collection itself.

### Create the Database 

In [8]:
client = MongoClient()

In [3]:
# client.list_database_names()

In [22]:
db = client.pak

lostPoets = db.lostPoets
collectionInfo = db.collectionInfo

**Info on Collection**

In [10]:
url = "https://api.opensea.io/api/v1/collection/lostpoets"

r = requests.request("GET", url)

r.json()

{'collection': {'editors': ['0xeaf54391793cc80de696d72713d7518c6190bfe0',
   '0x0239769a1adf4def9f07da824b80b9c4fcb59593',
   '0xe96a1b303a1eb8d04fb973eb2b291b8d591c8f72',
   '0x333601a803cac32b7d17a38d32c9728a93b422f4',
   '0x28679a1a632125fbbf7a68d850e50623194a709e',
   '0x92b381515bd4851faf3d33a161f7967fd87b1227',
   '0x700fe545742485732575a7245f558978adcc1ec4',
   '0xeeadc5701b5fd25fbcc7fbf04b048cdb1fff1c65',
   '0x50088f08381e2a04c6c485dab0eb03561a011c41',
   '0x889d43a6f9c2e48b913559a1333ead19e6444499',
   '0x4884eb76842f734e0de54d79ad45266072eb68e2',
   '0x9e26e77e3c494628467615b9ef5fc8f3c095cf89',
   '0x5e98629b10d2537988ff63ebb57df7378e08e373',
   '0x3e275ddf0b02dc4626deba238d175295064c0efe',
   '0xffb36e12031d07b4fe71c21cd7a01e29089d86d4',
   '0x2ce780d7c743a57791b835a9d6f998b15bbba5a4',
   '0xfdb16a768c4d41d7c0e1e5f5ce1d3c8234f6db0a',
   '0x836fd05a18dc13aa8885a718278e6a316ecca1ff',
   '0xd144054f09d17175d66aaa9ee73d7931c627fa9a',
   '0x931d24f3dede331bfd02a1aa855e9cbf7f0ea2

In [11]:
db.collectionInfo.insert_one(r.json())

In [12]:
client.list_database_names()

['admin', 'books', 'config', 'events', 'local', 'outings', 'pak']

In [13]:
cursor = db.collectionInfo.find({})
len(list(cursor))

1

**Collecting data on the items: lostPoets Collection** 

Message from OpenSea: The key is 00254e2f4e8944f187e78d64addffbfb.

In [17]:
#api_key = '00254e2f4e8944f187e78d64addffbfb'

First let's see what fields are available to us. 

In [12]:
url = "https://api.opensea.io/api/v1/assets?token_ids=1&asset_contract_address=0x4b3406a41399c7FD2BA65cbC93697Ad9E7eA61e5&order_direction=asc&offset=0"

response = requests.request("GET", url)

response.json()

{'assets': [{'id': 59080697,
   'token_id': '1',
   'num_sales': 0,
   'background_color': None,
   'image_url': 'https://lh3.googleusercontent.com/ulT6Iu_7798lvdulT253Ubtgoi3JYYnn5fo_tQ4A5l2NFFIkLAfqjRm5o8W9UMffMg8a6CMDmU7IZ9rFnPbSPnZ7YPdKBM8AnWMJ',
   'image_preview_url': 'https://lh3.googleusercontent.com/ulT6Iu_7798lvdulT253Ubtgoi3JYYnn5fo_tQ4A5l2NFFIkLAfqjRm5o8W9UMffMg8a6CMDmU7IZ9rFnPbSPnZ7YPdKBM8AnWMJ=s250',
   'image_thumbnail_url': 'https://lh3.googleusercontent.com/ulT6Iu_7798lvdulT253Ubtgoi3JYYnn5fo_tQ4A5l2NFFIkLAfqjRm5o8W9UMffMg8a6CMDmU7IZ9rFnPbSPnZ7YPdKBM8AnWMJ=s128',
   'image_original_url': 'https://d1xxei964ioe0z.cloudfront.net/full/a161f1a5f415f4e14c76105c15b190e15071c89be025845aad5bab3b25ebaccd.png',
   'animation_url': None,
   'animation_original_url': None,
   'name': 'I',
   'description': None,
   'external_link': None,
   'asset_contract': {'address': '0x4b3406a41399c7fd2ba65cbc93697ad9e7ea61e5',
    'asset_contract_type': 'non-fungible',
    'created_date': '202

Now let's get the data into our database.

In [25]:
url = "https://api.opensea.io/api/v1/assets"

for i in range(0, 1000): 
    query = {"asset_contract_address":"0x4b3406a41399c7FD2BA65cbC93697Ad9E7eA61e5",
             "order_direction":"asc",
             "offset":i*20,
             "limit":"20"}
    headers = {
        "Accept": "application/json",
        "X-API-KEY": "00254e2f4e8944f187e78d64addffbfb"
    }

    response = requests.request("GET", url, headers=headers, params=query)

    if response.status_code != 200:
        print('error')
        break

    poets = response.json()['assets']
    
    if (len(poets)) == 0: 
        print('no more poets here')
        break
    else: 
        print(f"At pass {i}, inserted {len(poets)} poets.")

    # storing data in MongoDB
    lostPoets.insert_many(poets)


At pass 0, inserted 20 poets.
At pass 1, inserted 20 poets.
At pass 2, inserted 20 poets.
At pass 3, inserted 20 poets.
At pass 4, inserted 20 poets.
At pass 5, inserted 20 poets.
At pass 6, inserted 20 poets.
At pass 7, inserted 20 poets.
At pass 8, inserted 20 poets.
At pass 9, inserted 20 poets.
At pass 10, inserted 20 poets.
At pass 11, inserted 20 poets.
At pass 12, inserted 20 poets.
At pass 13, inserted 20 poets.
At pass 14, inserted 20 poets.
At pass 15, inserted 20 poets.
At pass 16, inserted 20 poets.
At pass 17, inserted 20 poets.
At pass 18, inserted 20 poets.
At pass 19, inserted 20 poets.
At pass 20, inserted 20 poets.
At pass 21, inserted 20 poets.
At pass 22, inserted 20 poets.
At pass 23, inserted 20 poets.
At pass 24, inserted 20 poets.
At pass 25, inserted 20 poets.
At pass 26, inserted 20 poets.
At pass 27, inserted 20 poets.
At pass 28, inserted 20 poets.
At pass 29, inserted 20 poets.
At pass 30, inserted 20 poets.
At pass 31, inserted 20 poets.
At pass 32, insert

At pass 260, inserted 20 poets.
At pass 261, inserted 20 poets.
At pass 262, inserted 20 poets.
At pass 263, inserted 20 poets.
At pass 264, inserted 20 poets.
At pass 265, inserted 20 poets.
At pass 266, inserted 20 poets.
At pass 267, inserted 20 poets.
At pass 268, inserted 20 poets.
At pass 269, inserted 20 poets.
At pass 270, inserted 20 poets.
At pass 271, inserted 20 poets.
At pass 272, inserted 20 poets.
At pass 273, inserted 20 poets.
At pass 274, inserted 20 poets.
At pass 275, inserted 20 poets.
At pass 276, inserted 20 poets.
At pass 277, inserted 20 poets.
At pass 278, inserted 20 poets.
At pass 279, inserted 20 poets.
At pass 280, inserted 20 poets.
At pass 281, inserted 20 poets.
At pass 282, inserted 20 poets.
At pass 283, inserted 20 poets.
At pass 284, inserted 20 poets.
At pass 285, inserted 20 poets.
At pass 286, inserted 20 poets.
At pass 287, inserted 20 poets.
At pass 288, inserted 20 poets.
At pass 289, inserted 20 poets.
At pass 290, inserted 20 poets.
At pass 

We run into an error because not all the Origin Poets have been minted yet. Let's see how many we have at the end of this pass. 

In [26]:
cursor = db.lostPoets.find({})
len(list(cursor))

10020

In [27]:
lostPoets_data = lostPoets.find()
poets_df = pd.DataFrame(lostPoets_data)
poets_df.head(3)

,_id,id,token_id,num_sales,background_color,image_url,image_preview_url,image_thumbnail_url,image_original_url,animation_url,...,owner,sell_orders,creator,traits,last_sale,top_bid,listing_date,is_presale,transfer_fee_payment_token,transfer_fee
0,61b6bdd216629c8ab9d260d4,59080697,1,0,None,https://lh3.googleusercontent.com/ulT6Iu_7798l...,https://lh3.googleusercontent.com/ulT6Iu_7798l...,https://lh3.googleusercontent.com/ulT6Iu_7798l...,https://d1xxei964ioe0z.cloudfront.net/full/a16...,None,...,"{'user': {'username': 'pak'}, 'profile_img_url...",None,"{'user': {'username': 'pak'}, 'profile_img_url...","[{'trait_type': 'Named', 'value': 'True', 'dis...",None,None,None,False,None,None
1,61b6bdd216629c8ab9d260d5,59350999,2,0,None,https://lh3.googleusercontent.com/gtvCMMER8xEL...,https://lh3.googleusercontent.com/gtvCMMER8xEL...,https://lh3.googleusercontent.com/gtvCMMER8xEL...,https://d1xxei964ioe0z.cloudfront.net/full/622...,None,...,"{'user': {'username': None}, 'profile_img_url'...",None,"{'user': {'username': 'pak'}, 'profile_img_url...","[{'trait_type': 'Origin', 'value': '7THK', 'di...",None,None,None,False,None,None
2,61b6bdd216629c8ab9d260d6,59351004,3,0,None,https://lh3.googleusercontent.com/xgxBi9sL8ZOT...,https://lh3.googleusercontent.com/xgxBi9sL8ZOT...,https://lh3.googleusercontent.com/xgxBi9sL8ZOT...,https://d1xxei964ioe0z.cloudfront.net/full/304...,None,...,{'user': {'username': 'LordTruffington_Vault'}...,None,"{'user': {'username': 'pak'}, 'profile_img_url...","[{'trait_type': 'Origin', 'value': '6AF2', 'di...",None,None,None,False,None,None


Let's make sure we don't have any duplicates.  

In [28]:
poets_df.token_id.duplicated().sum()

0

In [30]:
#poets_df.to_csv("poets_df_Dec12.csv", index= False)